# 4-5 AutoGraph and tf.Module


There are three ways of constructing graph: static, dynamic and Autograph.

TensorFlow 2.X uses dynamic graph and Autograph.

Dynamic graph is easier for debugging with higher encoding efficiency, but with lower efficiency in execution.

Static graph has high efficiency in execution, but more difficult for debugging.

Autograph mechanism transforms dynamic graph into static graph, making allowance for both executing and encoding efficiencies.

There are certain rules for the code that is able to converted by Autograph, or it could result in failure or unexpected results.

The coding rules and the mechanisms of Autograph were introduced in the last sections.

In this section, we introduce constructing Autograph using `tf.Module`.



### 1. Introduction to Autograph and `tf.Module`


We mentioned that the definition of `tf.Variable` should be avoided inside the decorator `@tf.function`.

However, it would seem to be a imperfect leaked package if we define `tf.Variable` outside the function, since the function has outside dependency.

One of the simple solutions is: defining a class and place the definition of `tf.Variable` inside the initial method, and leave the other methods/implementation elsewhere.

After such an ingenious operation, it is so naturally as if the chronic constipation was cured by the best laxative.

The surprise is that TensorFlow providing us a base class `tf.Module` to get the above naturally. What's more, It is supposed to be inherited for constructing child classes to manage variables and other `Module` conveniently. And the most important that it allows us to save model through `tf.saved_model` and achieve cross-platform deployment. What a surprise!

In fact, `tf.keras.models.Model`, `tf.keras.layers.Layer` are both inherited from `tf.Module`. They provides the management to the variables and the referred sub-modules.

**We are able to develop arbitrary learning model (not only neural network) and implement cross-platform deployment through the packaging provided by `tf.Module` and the low-level APIs in TensorFlow.**


### 2. Packaging Autograph Using `tf.Module`


We define a simple function。

In [1]:
import tensorflow as tf 
x = tf.Variable(1.0,dtype=tf.float32)

# Use input_signature to limit the signature type of the input tensors with shape and dtype inside the decorator tf.function
@tf.function(input_signature=[tf.TensorSpec(shape = [], dtype = tf.float32)])    
def add_print(a):
    x.assign_add(a)
    tf.print(x)
    return(x)

In [2]:
add_print(tf.constant(3.0))
#add_print(tf.constant(3)) # Error: argument inconsistent with the tensor signature.

4


<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

Package using `tf.Module`.

In [3]:
class DemoModule(tf.Module):
    def __init__(self,init_value = tf.constant(0.0),name=None):
        super(DemoModule, self).__init__(name=name)
        with self.name_scope:  # Identical to: with tf.name_scope("demo_module")
            self.x = tf.Variable(init_value,dtype = tf.float32,trainable=True)

     
    @tf.function(input_signature=[tf.TensorSpec(shape = [], dtype = tf.float32)])  
    def addprint(self,a):
        with self.name_scope:
            self.x.assign_add(a)
            tf.print(self.x)
            return(self.x)


In [4]:
# Execute
demo = DemoModule(init_value = tf.constant(1.0))
result = demo.addprint(tf.constant(5.0))

6


In [5]:
# Browse all variables and trainable variables in the module
print(demo.variables)
print(demo.trainable_variables)


(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)
(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)


In [6]:
# Browse all sub-modules
demo.submodules

()

In [8]:
# Save the model using tf.saved_model and specify the method of cross-platform deployment.
tf.saved_model.save(demo,"./data/demo/1",signatures = {"serving_default":demo.addprint})

INFO:tensorflow:Assets written to: ./data/demo/1/assets


In [9]:
# Save the model using tf.saved_model and specify the method of cross-platform deployment.
tf.saved_model.save(demo,"./data/demo/1",signatures = {"serving_default":demo.addprint})


INFO:tensorflow:Assets written to: ./data/demo/1/assets


In [10]:
# Check the info of the model file. The info in the red rectangulars could be used during the deployment and the cross-platform usage.
!saved_model_cli show --dir ./data/demo/1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['a'] tensor_info:
        dtype: DT_FLOAT
        shape: ()
        name: serving_default_a:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: ()
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Defined Functions:
  Function Name: 'addprint'
    Option #1
      Callable wit


Check the graph in tensorboard, the module will be added with name `demo_module`, showing the hierarchy of the graph.

In [16]:
import datetime

# Creating log
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = './data/demomodule/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Start tracing of the Autograph
tf.summary.trace_on(graph=True, profiler=True) 

# Execute the Autograph
demo = DemoModule(init_value = tf.constant(0.0))
result = demo.addprint(tf.constant(5.0))

# Write the info of the graph into the log
with writer.as_default():
    tf.summary.trace_export(
        name="demomodule",
        step=0,
        profiler_outdir=logdir)
    

5


In [12]:
# Magic command to launch tensorboard in jupyter
%reload_ext tensorboard

In [13]:
from tensorboard import notebook
notebook.list() 

No known TensorBoard instances running.


In [18]:
notebook.start("--logdir ./data/demomodule/")


Besides using the child class of `tf.Module`, it is also possible to package through adding attribute to `tf.Module`.

In [19]:
mymodule = tf.Module()
mymodule.x = tf.Variable(0.0)

@tf.function(input_signature=[tf.TensorSpec(shape = [], dtype = tf.float32)])  
def addprint(a):
    mymodule.x.assign_add(a)
    tf.print(mymodule.x)
    return (mymodule.x)

mymodule.addprint = addprint

In [20]:
mymodule.addprint(tf.constant(1.0)).numpy()

1


1.0

In [21]:
print(mymodule.variables)

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>,)


In [22]:
# Save model using tf.saved_model
tf.saved_model.save(mymodule,"./data/mymodule",
    signatures = {"serving_default":mymodule.addprint})

# Load the model
mymodule2 = tf.saved_model.load("./data/mymodule")
mymodule2.addprint(tf.constant(5.0))

INFO:tensorflow:Assets written to: ./data/mymodule/assets
6


<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

### 3. `tf.Module` and `tf.keras.Model`，`tf.keras.layers.Layer`


The models and the layers in `tf.keras` are implemented through inheriting `tf.Module`. Both of them are able to manage variables and sub-modules.


In [23]:
import tensorflow as tf
from tensorflow.keras import models,layers,losses,metrics

In [24]:
print(issubclass(tf.keras.Model,tf.Module))
print(issubclass(tf.keras.layers.Layer,tf.Module))
print(issubclass(tf.keras.Model,tf.keras.layers.Layer))

True
True
True


In [25]:
tf.keras.backend.clear_session() 

model = models.Sequential()

model.add(layers.Dense(4,input_shape = (10,)))
model.add(layers.Dense(2))
model.add(layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.variables

[<tf.Variable 'dense/kernel:0' shape=(10, 4) dtype=float32, numpy=
 array([[-0.39357895,  0.17201114, -0.17684093, -0.2223306 ],
        [-0.5456597 , -0.61319715,  0.5453149 , -0.49931315],
        [ 0.4616909 , -0.3636119 ,  0.53100514, -0.10496271],
        [-0.15130019, -0.65385187, -0.5520497 ,  0.5473343 ],
        [-0.27018473, -0.28345183,  0.43904173,  0.2881096 ],
        [-0.6061774 ,  0.05067313,  0.31084698, -0.3930954 ],
        [ 0.24834347, -0.6037528 ,  0.56127477, -0.4349454 ],
        [ 0.35327137, -0.00703067,  0.4317726 ,  0.5719557 ],
        [ 0.3288405 ,  0.34653926, -0.46839088, -0.1573855 ],
        [-0.34582734, -0.07715958,  0.34878373,  0.47803998]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[-0.91984844, -0.4301288 ],
        [-0.35146165, -0.9562588 ],
        [ 0.38753748, -0.17635036],
        [-

In [27]:
model.layers[0].trainable = False # Freeze the variables in layer 0, make it untrainable.
model.trainable_variables

[<tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[-0.91984844, -0.4301288 ],
        [-0.35146165, -0.9562588 ],
        [ 0.38753748, -0.17635036],
        [-0.19446874, -0.00458908]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'dense_2/kernel:0' shape=(2, 1) dtype=float32, numpy=
 array([[-1.2840478 ],
        [-0.36043143]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [28]:
model.submodules

(<tensorflow.python.keras.engine.input_layer.InputLayer at 0x7feee3baa150>,
 <tensorflow.python.keras.layers.core.Dense at 0x7feee47bf790>)

In [29]:
model.layers

In [30]:
print(model.name)
print(model.name_scope())

sequential
sequential
